In [1]:
# bert task2
# 2021-1-23 start test

# 挂载谷歌网盘
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
%cd "/content/drive/My Drive/SemEval-2021-task6(1-23)/task2"
%ls

/content/drive/My Drive/SemEval-2021-task6(1-23)/task2
checkpoint/  metric.py      Save_result.py    task_two_section.ipynb
DataSet/     PreProcess.py  task2-bert.ipynb  training_set_task2.txt
log/         __pycache__/   task2-output.txt


In [ ]:
!pip install transformers==4.0.0
!pip install tensorflow==2.3.0

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer,OneHotEncoder  # 多标签编码
from transformers import BertTokenizerFast,AlbertTokenizerFast
import tensorflow as tf

tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')
#tokenizer = AlbertTokenizerFast.from_pretrained('albert-base-v2')

In [5]:
# 读取train data_id
def load_data_ids(file):
    import sys
    import json
    try:
        with open(file, "r", encoding="utf8") as f:
            jsonobj = json.load(f)
    except:
        sys.exit("ERROR: cannot load json file")

    id = []
    for example in jsonobj:
        id.append(example['id'])

    return id

def get_technique_list():
    # 读取techiques列表
    techiques_list_filename = '../techniques_list_task1-2.txt'
    with open(techiques_list_filename, "r", encoding='utf8') as f:
        techiques_list = [line.rstrip() for line in f.readlines() if len(line) > 2]
    return techiques_list

def load_data(path,technique_list):
    df = pd.read_csv(path,header=0)
    text = list(df['text'])
    label = []
    for technique in technique_list:   # 返回20种标签的矩阵
        label.append(list(df[technique]))
    return text,label

In [6]:
techniques = get_technique_list()

In [29]:
import copy

train_ids = load_data_ids(file = "../data/training_set_task2.txt")

train_texts,train_tags = [],[]
for i in train_ids:
    token,label = load_data('./DataSet/train_BIO/'+str(i)+'.csv',techniques)
    train_texts.append(token)
    train_tags.append(label)


dev_ids = load_data_ids(file = "../data/dev_set_task2.txt")
dev_texts,dev_tags = [],[]
for i in dev_ids:
    token,label = load_data('./DataSet/dev_BIO/'+str(i)+'.csv',techniques)
    dev_texts.append(token)
    dev_tags.append(label)

In [30]:
print(len(dev_texts),len(train_texts))

63 688


In [31]:
print(len(train_tags))
print(len(train_texts[62]))
print(len(train_tags[62][8]))
print(train_texts)
print(train_tags[0])

688
25
25
[['THERE', 'ARE', 'ONLY', 'TWO', 'GENDERS', 'FEMALE', 'MALE'], ['This', 'is', 'not', 'an', 'accident!'], ['SO', 'BERNIE', 'BROS', "HAVEN'T", 'COMMITTED', 'VIOLENCE', 'EH?', 'POWER', 'COMES', 'FROM', 'THE', 'BARREL', 'OF', 'A', 'GUN,', 'COMRADES.', 'WHAT', 'ABOUT', 'THE', 'ONE', 'WHO', 'SHOT', 'CONGRESSMAN', 'SCALISE', 'OR', 'THE', 'DAYTON', 'OHIO', 'MASS', 'SHOOTER?'], ['PATHETIC', 'The', 'Cowardly', 'Asshole', 'Weak', 'Failure!!!', 'Impeach', 'it', 'again!!!', '#', '45', 'the', 'coward', 's', 'afraid', 'of', 'strong', 'women!!', 'POLITICUSUSA.COM', 'Childish', 'Trump', "Won't", 'Meet', 'With', 'Pelosi', 'On', 'Coronavirus', 'Because', 'He', "Doesn't", 'Like', 'Her'], ['WHO', 'TRUMP', 'REPRESENTS', 'WHO', 'DEMOCRATS', 'REPRESENT'], ['Our', 'elders', 'were', 'called', 'to', 'war', 'to', 'save', 'lives.', 'We', 'are', 'being', 'called', 'to', 'sit', 'on', 'the', 'couch', 'to', 'save', 'theirs.', 'We', 'can', 'do', 'this.'], ['Adolf', 'Hitler', '•', 'Believed', 'state', 'power',

In [ ]:
for tech in techniques:
    for i in train_tags:
        for j in i:
            if ('B-' + "Loaded Language") in j:
                print(j)

    print("dev")
    for i in dev_tags:
        for j in i:
            if ('B-' + "Loaded Language") in j:
                print(j)

In [33]:
print(techniques)

['Appeal to authority', 'Appeal to fear/prejudice', 'Black-and-white Fallacy/Dictatorship', 'Causal Oversimplification', 'Doubt', 'Exaggeration/Minimisation', 'Flag-waving', 'Glittering generalities (Virtue)', 'Loaded Language', "Misrepresentation of Someone's Position (Straw Man)", 'Name calling/Labeling', 'Obfuscation, Intentional vagueness, Confusion', 'Presenting Irrelevant Data (Red Herring)', 'Reductio ad hitlerum', 'Repetition', 'Slogans', 'Smears', 'Thought-terminating cliché', 'Whataboutism', 'Bandwagon']


In [34]:
print(len(dev_tags))
print(len(dev_texts[62]))
print(len(dev_tags[62][8]))
print(dev_texts)
print(dev_tags[0])

63
27
27
[['*President*', 'Biden?', 'Please,', 'no.'], ['JOE', 'VERSUS', 'THE', 'VOLCANIC', 'KREMLIN', 'DON', '"WILL', 'YOU', 'SHUT', 'UP,', 'MAN?"', 'LORD', 'OF', 'THE', 'LIES'], ['ANTI-VAXXERS', 'BE', 'LIKE...', 'HANG', 'ON', 'A', 'SEC', '-', 'JUST', 'COLLECTING', 'MY', 'FACTS'], ['VIRUS', 'BINGO', 'FREE', '32', 'SPACE'], ['Never', 'thought', "l'd", 'die', 'fighting', 'IRRESPONSIBLY', 'RUSHED', 'VACCINES', 'side', 'by', 'side', 'with', 'an', 'ANTIVAXXER', 'What', 'about', 'side', 'by', 'side', 'with', 'a', 'friend?', "DON'T", 'YOU', 'HAVE', 'A', "KID'S", 'FUNERAL', 'TO', 'ATTEND', 'OR', 'SOMETHING?'], ['Anti-Vaxers', 'keep', 'saying', 'dumb', 'things,', 'like', '"Vaccines', "aren't", '100%', 'effective,', 'so', 'they', 'are', 'useless!"', 'Parachutes', "aren't", '100%', 'effective', 'either,', 'but', 'wearing', 'one', 'is', 'a', 'damn', 'good', 'idea...'], ['Definition', 'Of', 'Stupid:', 'Knowing', 'The', 'Truth,', 'Seeing', 'The', 'Truth,', 'but', 'still', 'Believing', 'in', 'TRUDEA

In [35]:
val_texts = dev_texts
val_tags = dev_tags
print(len(val_texts),len(val_tags))

63 63


In [36]:
# 标签的映射

unique_tags = ['P']
for tech in techniques:
    #unique_tags.append('B'+'-'+tech)
    unique_tags.append('I'+'-'+tech)
    unique_tags.append('O'+'-'+tech)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

print(tag2id)
id_tags = []
for key in id2tag:
  id_tags.append(str(key))
#print(id_tags)
# print(mlb.fit_transform(['23'])[0])
print(tf.one_hot(tag2id["I-Misrepresentation of Someone's Position (Straw Man)"],depth = len(id_tags)))

{'P': 0, 'I-Appeal to authority': 1, 'O-Appeal to authority': 2, 'I-Appeal to fear/prejudice': 3, 'O-Appeal to fear/prejudice': 4, 'I-Black-and-white Fallacy/Dictatorship': 5, 'O-Black-and-white Fallacy/Dictatorship': 6, 'I-Causal Oversimplification': 7, 'O-Causal Oversimplification': 8, 'I-Doubt': 9, 'O-Doubt': 10, 'I-Exaggeration/Minimisation': 11, 'O-Exaggeration/Minimisation': 12, 'I-Flag-waving': 13, 'O-Flag-waving': 14, 'I-Glittering generalities (Virtue)': 15, 'O-Glittering generalities (Virtue)': 16, 'I-Loaded Language': 17, 'O-Loaded Language': 18, "I-Misrepresentation of Someone's Position (Straw Man)": 19, "O-Misrepresentation of Someone's Position (Straw Man)": 20, 'I-Name calling/Labeling': 21, 'O-Name calling/Labeling': 22, 'I-Obfuscation, Intentional vagueness, Confusion': 23, 'O-Obfuscation, Intentional vagueness, Confusion': 24, 'I-Presenting Irrelevant Data (Red Herring)': 25, 'O-Presenting Irrelevant Data (Red Herring)': 26, 'I-Reductio ad hitlerum': 27, 'O-Reductio 

In [37]:
def encode_single_tag(tag, offset_mapping):
    # 返回单个句子及对应标签的编码
    labels = tag
    #m = mlb.fit_transform(str(tag2id['P']))[0]
    m = tf.one_hot(tag2id['P'],depth = len(id_tags))
    encoding = [m]*len(offset_mapping)
    i = 0
    for label in labels:
        while i < len(offset_mapping):
            if offset_mapping[i][0] == 0 and offset_mapping[i][1]!=0:
                #print(label)
                #temp = mlb.fit_transform(str(tag2id[label]))[0] # 当前词语的编码
                temp = tf.one_hot(tag2id[label],depth = len(id_tags))
                encoding[i] = temp
                for j in range(i+1,len(offset_mapping)):
                    if offset_mapping[j][0] == 0 and offset_mapping[i][1]!=0: # 若已经到下一个词，即(0,x)
                        break
                    elif offset_mapping[j][0] !=0 and offset_mapping[j][1] !=0: # 若为（n,m）则表示还是该词
                        encoding[j] = temp
                        i = j
                i = i + 1
                break
            i = i + 1
    return encoding

def encode_labels(tags,offset):
    # 返回所有句子和对应标签的编码
    result = []
    for i in range(len(tags)): #607
        encoded_labels = []
        for j in range(len(tags[0])): # 20
            encoding = encode_single_tag(tags[i][j],offset[i])
            encoded_labels.append(encoding)
        result.append(encoded_labels)
    return result

In [38]:
print(len(train_texts),len(val_texts))

688 63


In [39]:
train_encodings = tokenizer(train_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
val_encodings = tokenizer(val_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
print(len(train_texts[62]))
print(train_encodings['input_ids'][0])

25
[101, 7462, 16941, 22133, 2036, 21748, 2162, 3663, 157, 2924, 2346, 25075, 16769, 9637, 1708, 143, 15577, 12507, 2036, 9960, 17516, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [40]:
train_labels = encode_labels(train_tags,train_encodings.offset_mapping)
val_labels = encode_labels(val_tags,val_encodings.offset_mapping)

In [41]:
print(len(train_labels))
print(len(val_labels))
print(len(train_labels[62]))
print(len(train_labels[62][8]))
print(train_labels[63][8][0])
for i in range(len(train_labels)):
  print(np.argmax(train_labels[i][18],-1))
  print(train_tags[i][18])

[ 0 38 38 38 38 38 38 38 38 38 38 38 38 38 38 38 38 38 38 38 38 38 38 38
 38 38 38 38 38 38 38 38 38 38 38 38 38 38 38 38 38 38 38 38 38 38 38 38
 38 38 38 38 38 38 38 38 38 38 38 38 38 38 38 38 38 38 38 38 38 38 38 38
 38 38 38 38 38 38 38 38 38 38 38 38 38 38 38  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
['O-Whataboutism', 'O-Whataboutism', 'O-Whataboutism', 'O-Whataboutism', 'O-Whataboutism', 'O-Whataboutism', 'O-Whataboutism', 'O-Whataboutism', 'O-Whataboutism', 'O-Whataboutism', 'O-Whataboutism', 'O-Whataboutism', 'O-Whataboutism', 'O-Whataboutism', 'O-Whataboutism', 'O-Whataboutism', 'O-Whataboutism', 'O-Whataboutism', 'O-Whataboutism', 'O-Whataboutism', 'O-Whataboutism', 'O-Whataboutism', 'O-Whataboutism', 'O-Whataboutism', 'O-

ValueError: ignored

In [42]:
train_encodings.pop("offset_mapping") # we don't want to pass this to the model
val_encodings.pop("offset_mapping")

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))

In [43]:
from tensorflow.keras.layers import Dense,TimeDistributed
from tensorflow.keras import Model

class multi_task_dense(Model):
    def __init__(self,initializer,num_label,num_dense=20):
        super(multi_task_dense, self).__init__()
        self.dense_list = []
        for i in range(num_dense):
            dense = TimeDistributed(Dense(num_label, kernel_initializer=initializer, activation='softmax'))
            self.dense_list.append(dense)

    def call(self, inputs):
        y = []
        for i in range(20):
            x = self.dense_list[i](inputs)
            y.append(x)
        y = tf.stack(y,axis=1)
        return y

In [44]:
from transformers import AlbertConfig, TFAlbertModel, BertConfig, TFBertModel

def create_model(num_label):
    # Albert层
    #config = AlbertConfig.from_pretrained('albert-base-v2')
    #bert_layer = TFAlbertModel.from_pretrained('albert-base-v2',config=config)
    
    config = BertConfig.from_pretrained('bert-base-uncased')
    bert_layer = TFBertModel(config=config)
    initializer = tf.keras.initializers.TruncatedNormal(config.initializer_range)

    # 构建bert输入
    input_ids = tf.keras.Input(shape=(config.max_position_embeddings,), dtype='int32')
    token_type_ids = tf.keras.Input(shape=(config.max_position_embeddings,), dtype='int32')
    attention_mask = tf.keras.Input(shape=(config.max_position_embeddings,), dtype='int32')
    inputs = {'input_ids': input_ids, 'token_type_ids': token_type_ids, 'attention_mask': attention_mask}

    # bert的输出
    bert_output = bert_layer(inputs)
    print(bert_output)
    hidden_states = bert_output[0]
    hidden_dropout = tf.keras.layers.Dropout(0.1)(hidden_states)

    #d1 = TimeDistributed(Dense(4,kernel_initializer=initializer,activation='softmax'))(hidden_dropout)
    dense = multi_task_dense(initializer,num_label = num_label)(hidden_dropout)
    output = dense
    print(output)
    model = tf.keras.Model(inputs=inputs, outputs=output)
    # print(config.max_position_embeddings)
    model.summary()
    return model


In [45]:
model = create_model(len(unique_tags))

TFBaseModelOutputWithPooling(last_hidden_state=<tf.Tensor 'tf_bert_model_1/bert/encoder/layer_._11/output/LayerNorm/batchnorm/add_1:0' shape=(None, 512, 768) dtype=float32>, pooler_output=<tf.Tensor 'tf_bert_model_1/bert/pooler/dense/Tanh:0' shape=(None, 768) dtype=float32>, hidden_states=None, attentions=None)
Tensor("multi_task_dense_1/stack:0", shape=(None, 20, 512, 41), dtype=float32)
Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 512)]        0                                            
___________________________________________________________________________________________

In [46]:
from transformers import TFBertForTokenClassification
from metric import Metrics, f1
import os
#m = TFBertForTokenClassification.from_pretrained('bert-base-cased')
# 断点续训
checkpoint_save_path = "./checkpoint/Bert_Crossentropy_IO/seq_tagging.ckpt"
if os.path.exists(checkpoint_save_path + '.index'):
    print('===============Load Model=================')
    model.load_weights(checkpoint_save_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_save_path,
                                                  monitor='val_f1',
                                                  mode='max', verbose=1,
                                                  save_weights_only=True,
                                                  save_best_only=True, )


# 训练模型
optimizer = tf.keras.optimizers.Adam(lr=5e-7, epsilon=1e-8)
loss = tf.keras.losses.CategoricalCrossentropy()  # 二进制交叉熵
metric = tf.keras.metrics.CategoricalAccuracy()
model.compile(optimizer=optimizer, loss=loss, metrics=[f1])

===============Load Model=================


In [ ]:
print("===============Start training=================")
history = model.fit(train_dataset.shuffle(1000).batch(16), validation_data=val_dataset.batch(16),epochs=500,
                        batch_size=16,
                        callbacks=cp_callback
                        )

In [ ]:
model.load_weights(checkpoint_save_path)
y_pred = model.predict(val_dataset.batch(16))
print(y_pred.shape)

In [ ]:
print(np.argmax(y_pred,-1))

In [ ]:

r = np.argmax(y_pred,-1)
print(r.shape)
for i in r[0]:
  print(i)

In [50]:
import copy
# 预测得到结果后对标签解码
def decode_sigle_tag(pred,offset):
    # 解码单个句子标签
    predict = copy.deepcopy(pred)
    for i in range(len(predict)):
        if offset[i][0] == 0 and offset[i][1] == 0: # 删除填充词
            predict[i] = 999 # 999标记为删除该项
        elif offset[i][0] !=0 and offset[i][1] != 0: # 删除分解词（只保留第一个）
            predict[i] = 999
    j = 0
    for i in range(len(predict)):
        if predict[j] == 999:
            predict.pop(j)
        else:
            predict[j] = id2tag[predict[j]]
            j = j + 1
    return predict

def decode_labels(pred,offset):
    # 返回所有句子和对应标签的编码
    result = []
    for i in range(len(pred)): #607
        decoded_labels = []
        for j in range(len(pred[0])): # 20
            decoding = decode_sigle_tag(list(pred[i][j]),offset[i])
            #print(decoding)
            decoded_labels.append(decoding)
        result.append(decoded_labels)
    return result

In [51]:
train_encodings = tokenizer(train_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
val_encodings = tokenizer(val_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
output = decode_labels(list(r),val_encodings.offset_mapping)

In [52]:
for i in range(len(output)):
    if len(output[i][8])!=len(val_tags[i][8]):
        print(i,len(output[i][8]),len(val_tags[i][8]))

In [ ]:
for i in range(len(output)):
  for j in range(len(output[i])):
    print("%d\n"%i)
    #print(len(output[i][8]))
    #print(len(val_tags[i][8]))
    print("true:",val_tags[i][j])
    #print(len(val_labels[i][8]))
    print("pred:",output[i][j])

In [ ]:
!ls

checkpoint  log        PreProcess.py  Save_result.py	task_two_section.ipynb
DataSet     metric.py  __pycache__    task2-bert.ipynb	training_set_task2.txt


In [ ]:
# 将结果按id保存为csv
for k in range(len(output)):
    print("\n",dev_ids[k])
    df = pd.DataFrame()
    df['text'] = val_texts[k]
    for i in range(len(techniques)):
        print(techniques[i])
        print(dev_texts[k])
        print(output[k][i])
        df[techniques[i]] = output[k][i]
    path = './DataSet/dev_result/'+str(dev_ids[k])+'.csv'
    df.to_csv(path,index=False,encoding='utf-8')

In [57]:
!ls

checkpoint  PreProcess.py   task2-bert.ipynb	       training_set_task2.txt
DataSet     __pycache__     task2-output.txt
log	    save_dev.py     task2-test_set-output.txt
metric.py   Save_result.py  task_two_section.ipynb


In [58]:
!python3 save_dev.py

tech: Smears
start: OR
end: OR
{'start': 175, 'end': 188, 'technique': 'Smears', 'text_fragment': 'OR SOMETHING?'}
Save Complete
tech: Loaded Language
start: damn
end: damn
{'start': 160, 'end': 177, 'technique': 'Loaded Language', 'text_fragment': 'damn good idea...'}
Save Complete
tech: Name calling/Labeling
start: damn
end: damn
{'start': 160, 'end': 177, 'technique': 'Name calling/Labeling', 'text_fragment': 'damn good idea...'}
Save Complete
tech: Smears
start: 100%
end: damn
{'start': 59, 'end': 177, 'technique': 'Smears', 'text_fragment': '100% effective, so they are useless!"\n\nParachutes aren\'t 100% effective either, but\nwearing one is a damn good idea...'}
Save Complete
tech: Loaded Language
start: VILLAINS
end: VILLAINS
{'start': 98, 'end': 164, 'technique': 'Loaded Language', 'text_fragment': 'VILLAINS ANNOUNCE THEIR MURDEROUS PLANS IN TED TALK PRESENTATIONS.'}
Save Complete
tech: Smears
start: VACCINES!
end: TED
{'start': 70, 'end': 164, 'technique': 'Smears', 'text_fra

In [ ]:
# 在test集上预测
def load_test_data(file):
    import sys
    import json
    try:
        with open(file, "r", encoding="utf8") as f:
            jsonobj = json.load(f)
    except:
        sys.exit("ERROR: cannot load json file")

    id,texts = [],[]
    for example in jsonobj:
        id.append(example['id'])
        texts.append(example["text"])
    return id,texts

path = "../data/test_set_task2.txt"
test_id,texts = load_test_data(path)
test_texts = []
for text in texts:
    temp = text.split()
    test_texts.append(temp)
for i in test_texts:
    print(i)

In [61]:
test_encodings = tokenizer(test_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)

test_encodings.pop("offset_mapping") # we don't want to pass this to the model


test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
))

In [62]:
y_pred = model.predict(test_dataset.batch(16))
print(y_pred.shape)

(200, 20, 134, 41)


In [63]:
r = np.argmax(y_pred,-1)
print(r.shape)
for i in r[0]:
  print(i)

(200, 20, 134)
[0 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 4 4 4 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 6 6 6 6 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 8 8 8 8 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0

In [64]:
test_encodings = tokenizer(test_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
output = decode_labels(list(r),test_encodings.offset_mapping)

In [ ]:
for i in range(len(output)):
  for j in range(len(output[i])):
    print("%d\n"%i)
    #print(len(output[i][8]))
    #print(len(val_tags[i][8]))
    #print("true:",val_tags[i][j])
    #print(len(val_labels[i][8]))
    print("pred:",output[i][j])

In [66]:
!ls

checkpoint  PreProcess.py   task2-bert.ipynb	       task_two_section.ipynb
DataSet     __pycache__     task2-dev_set-output.txt   training_set_task2.txt
log	    save_dev.py     task2-output.txt
metric.py   Save_result.py  task2-test_set-output.txt


In [67]:
# 将结果按id保存为csv
for k in range(len(output)):
    print("\n",test_id[k])
    df = pd.DataFrame()
    df['text'] = test_texts[k]
    for i in range(len(techniques)):
        print(techniques[i])
        print(len(test_texts[k]))
        print(len(output[k][i]))
        df[techniques[i]] = output[k][i]
    # path = './DataSet/test_result/'+str(test_id[k])+'.csv'
    # df.to_csv(path,index=False,encoding='utf-8')

流式输出内容被截断，只能显示最后 5000 行内容。
30
Glittering generalities (Virtue)
30
30
Loaded Language
30
30
Misrepresentation of Someone's Position (Straw Man)
30
30
Name calling/Labeling
30
30
Obfuscation, Intentional vagueness, Confusion
30
30
Presenting Irrelevant Data (Red Herring)
30
30
Reductio ad hitlerum
30
30
Repetition
30
30
Slogans
30
30
Smears
30
30
Thought-terminating cliché
30
30
Whataboutism
30
30
Bandwagon
30
30

 942_batch_2
Appeal to authority
34
34
Appeal to fear/prejudice
34
34
Black-and-white Fallacy/Dictatorship
34
34
Causal Oversimplification
34
34
Doubt
34
34
Exaggeration/Minimisation
34
34
Flag-waving
34
34
Glittering generalities (Virtue)
34
34
Loaded Language
34
34
Misrepresentation of Someone's Position (Straw Man)
34
34
Name calling/Labeling
34
34
Obfuscation, Intentional vagueness, Confusion
34
34
Presenting Irrelevant Data (Red Herring)
34
34
Reductio ad hitlerum
34
34
Repetition
34
34
Slogans
34
34
Smears
34
34
Thought-terminating cliché
34
34
Whataboutism
34
34
Bandwago

In [68]:
!ls

checkpoint  PreProcess.py   task2-bert.ipynb	       task_two_section.ipynb
DataSet     __pycache__     task2-dev_set-output.txt   training_set_task2.txt
log	    save_dev.py     task2-output.txt
metric.py   Save_result.py  task2-test_set-output.txt


In [69]:
!python3 Save_result.py

tech: Smears
start: U.S.
end: IN
{'start': 69, 'end': 148, 'technique': 'Smears', 'text_fragment': 'U.S. COUNTIES\n\nYET SUPPOSEDLY WON THE ELECTION WITH THE MOST VOTES IN HISTORY\n\n'}
Save Complete
tech: Loaded Language
start: destroyed
end: gun
{'start': 142, 'end': 309, 'technique': 'Loaded Language', 'text_fragment': 'destroyed family. \nNever forget, Nazis tore down statues.\nBanned free speech.\nBlamed economic hardships on one group of people. Instituted gun control.\nSound Familiar?'}
Save Complete
tech: Smears
start: victims
end: control.
{'start': 79, 'end': 309, 'technique': 'Smears', 'text_fragment': 'victims prior to being executed, 1945. \nEach ring represents a destroyed family. \nNever forget, Nazis tore down statues.\nBanned free speech.\nBlamed economic hardships on one group of people. Instituted gun control.\nSound Familiar?'}
Save Complete
tech: Smears
start: AND
end: KNOW
{'start': 70, 'end': 159, 'technique': 'Smears', 'text_fragment': "AND IT HAS MILLIONS OF\nC